In [3]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

In [4]:
# Importer les bibliothèques nécessaires
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [5]:
from sklearn import preprocessing
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from src.analysis.analysis import   main_analysis
from src.tools.preprocess import preprocess_data
import joblib
from sklearn.linear_model import LogisticRegression

In [6]:
%pip install xgboost
%pip install imbalanced-learn
import xgboost as xgb

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
from shapely.wkt import loads
data = pd.read_csv('../data/processed/data.csv')
data['area']= data["geometry"].apply( lambda polygon_wkt : loads(polygon_wkt).area)
#data['length']=  data["geometry"].apply(lambda polygon_wkt : loads(polygon_wkt).length)
#data['num_vertices']=  data["geometry"].apply(lambda polygon_wkt : loads(polygon_wkt).coords)
data['centroid_x']=  data["geometry"].apply(lambda polygon_wkt : loads(polygon_wkt).centroid.x)
data['centroid_y']=  data["geometry"].apply(lambda polygon_wkt : loads(polygon_wkt).centroid.y)
data.head()

,Unnamed: 0,FarmID,category,Crop,State,District,Sub-District,SDate,HDate,CropCoveredArea,...,tif_path,ndvi,evi,ndwi,gndvi,savi,msavi,area,centroid_x,centroid_y
0,0,1326576,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-25 0:00:00,2024-04-14 0:00:00,97,...,downloads/20240412T050649_20240412T051727_T43Q...,0.100756,-0.793684,8.488031,0.127153,0.151125,4232.596191,9.956528e-07,78.181592,17.978863
1,1,1326577,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-11-13 0:00:00,2024-04-26 0:00:00,82,...,downloads/20240425T051649_20240425T052817_T43Q...,0.188090,0.564248,11.493762,0.187815,0.282110,3249.392822,1.497475e-06,78.175606,17.980966
2,2,1326578,Healthy,Paddy,Telangana,Medak,Kulcharam,2023-12-19 0:00:00,2024-04-28 0:00:00,92,...,downloads/20240427T050651_20240427T051439_T43Q...,0.206596,-1.456745,10.166884,0.206553,0.309869,3741.956055,9.668150e-07,78.169207,17.976084
3,3,1331836,Diseased,Paddy,Telangana,Medak,Kulcharam,2023-02-11 0:00:00,2024-11-04 0:00:00,91,...,downloads/20241103T050951_20241103T052004_T43Q...,0.206250,2.206184,10.446196,0.220995,0.309350,3673.396729,1.186327e-06,78.169059,17.974588
4,4,1326579,Diseased,Paddy,Telangana,Medak,Kulcharam,2023-12-12 0:00:00,2024-05-19 0:00:00,94,...,downloads/20240517T050651_20240517T051655_T43Q...,0.179721,-0.743638,8.665929,0.160657,0.269563,4227.338379,5.810344e-07,78.172695,17.969159


In [8]:
data_clean, anomalies = preprocess_data(data[data["dataset"]=="train"])

/home/paul/code/python/zindi/AAIS-AI/zindi_telangana_crop_health_challenge/src/tools/preprocess.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3943.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_clean.loc[non_null_mask & outlier_mask, col] = clean_mean
/home/paul/code/python/zindi/AAIS-AI/zindi_telangana_crop_health_challenge/src/tools/preprocess.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1301674.4039252088' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_clean.loc[non_null_mask & outlier_mask, col] = clean_mean
/home/paul/code/python/zindi/AAIS-AI/zindi_telangana_crop_health_challenge/src/tools/preprocess.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future ve

In [9]:
feature_columns = [
    'CropCoveredArea', 'CHeight', 'IrriCount', 'WaterCov', 'ExpYield',
    'ndvi', 'evi', 'ndwi', 'gndvi', 'savi', 'msavi', 'SDate_year', 'SDate_month',
       'SDate_day', 'HDate_year', 'HDate_month', 'HDate_day','centroid_y', 'centroid_x','area'
]

X = data_clean[feature_columns]
y = data_clean['category']
enc = preprocessing.OneHotEncoder()
le = LabelEncoder()
y = le.fit_transform(y)
for i in range(4):
    print(f"classe {i} : {(y==i).sum()/y.sum():2.2%}")
le.classes_

classe 0 : 5.42%
classe 1 : 74.40%
classe 2 : 5.55%
classe 3 : 4.84%


array(['Diseased', 'Healthy', 'Pests', 'Stressed'], dtype=object)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
target_names = le.classes_
# Définir les paramètres du modèle
params = {
    'objective': 'multi:softmax',  # Pour la classification multi-classes
    'num_class': 4,                 # 3 classes pour Iris
    'max_depth': 200,                 # Profondeur maximale des arbres
    'eta': 0.1,                     # Taux d'apprentissage
    'subsample': 0.8,               # Taux d'échantillonnage
    'colsample_bytree': 0.8,        # Taux d'échantillonnage des colonnes
    'eval_metric': 'mlogloss'       # Fonction de perte pour l'évaluation
}

# Entraîner le modèle
num_round = 1000  # Nombre d'itérations
bst = xgb.train(params, dtrain, num_round)

# Faire des prédictions
y_pred = bst.predict(dtest)

# Évaluer le modèle
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print(classification_report(y_test, y_pred, target_names=target_names))

Accuracy: 0.84
              precision    recall  f1-score   support

    Diseased       0.50      0.03      0.05        36
     Healthy       0.84      1.00      0.91       659
       Pests       0.00      0.00      0.00        40
    Stressed       0.00      0.00      0.00        54

    accuracy                           0.84       789
   macro avg       0.33      0.26      0.24       789
weighted avg       0.72      0.84      0.76       789



/home/paul/.cache/pypoetry/virtualenvs/telangana-crop-health-challenge-Bgz_xKMH-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/paul/.cache/pypoetry/virtualenvs/telangana-crop-health-challenge-Bgz_xKMH-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/paul/.cache/pypoetry/virtualenvs/telangana-crop-health-challenge-Bgz_xKMH-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and bein

In [42]:
def apply_geometry(data):
    data['area'] = data["geometry"].apply(
    lambda polygon_wkt: loads(polygon_wkt).area)
    data['centroid_x'] = data["geometry"].apply(
        lambda polygon_wkt: loads(polygon_wkt).centroid.x)
    data['centroid_y'] = data["geometry"].apply(
        lambda polygon_wkt: loads(polygon_wkt).centroid.y)
    data.head()
    return data

In [ ]:
test_encoded = pd.read_csv("../data/processed/data.csv")
test_encoded = apply_geometry(test_encoded)
test_encoded = test_encoded[test_encoded["dataset"]=="test"]
test_encoded['FarmID'] = test_encoded["FarmID"].astype(str)
test_encoded.drop(columns=["category"], inplace=True)
test_encoded, anomalies = preprocess_data(test_encoded)

feature_columns = [
    'CropCoveredArea', 'CHeight', 'IrriCount', 'WaterCov', 'ExpYield',
    'ndvi', 'evi', 'ndwi', 'gndvi', 'savi', 'msavi', 'SDate_year', 'SDate_month',
       'SDate_day', 'HDate_year', 'HDate_month', 'HDate_day','centroid_y', 'centroid_x','area'
]
X_test_submission =test_encoded[feature_columns]
X_test_submission = scaler.transform(X_test_submission)


dtest = xgb.DMatrix(X_test_submission)

test_predictions = bst.predict(dtest)
test_predictions = test_predictions.astype(int)

test_predictions_decoded = le.inverse_transform(test_predictions)

sample_submission = pd.DataFrame()
sample_submission['ID'] = test_encoded["FarmID"].astype(str)  # Convertir en chaîne de caractères
sample_submission['Target'] = test_predictions_decoded

# Aperçu du DataFrame
sample_submission.head()
# Sauvegarder le fichier de soumission
sample_submission.to_csv('submission_3.csv', index=False)

['Diseased' 'Healthy' 'Pests']


/home/paul/code/python/zindi/AAIS-AI/zindi_telangana_crop_health_challenge/src/tools/preprocess.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '9246.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_clean.loc[non_null_mask & outlier_mask, col] = clean_mean
/home/paul/code/python/zindi/AAIS-AI/zindi_telangana_crop_health_challenge/src/tools/preprocess.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '77.80242825607064' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_clean.loc[non_null_mask & outlier_mask, col] = clean_mean
/home/paul/code/python/zindi/AAIS-AI/zindi_telangana_crop_health_challenge/src/tools/preprocess.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future ver

In [12]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
target_names = le.classes_
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)

# Ajuster et transformer les données d'entraînement
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Créer un DMatrix pour XGBoost avec les données rééchantillonnées
dtrain_resampled = xgb.DMatrix(X_resampled, label=y_resampled)
dtest = xgb.DMatrix(X_test, label=y_test)

# Définir les paramètres du modèle
params = {
    'objective': 'multi:softmax',  # Pour la classification multi-classes
    'num_class': 4,                 # 3 classes pour Iris
    'max_depth': 200,                 # Profondeur maximale des arbres
    'eta': 0.1,                     # Taux d'apprentissage
    'subsample': 0.8,               # Taux d'échantillonnage
    'colsample_bytree': 0.8,        # Taux d'échantillonnage des colonnes
    'eval_metric': 'mlogloss'       # Fonction de perte pour l'évaluation
}

# Entraîner le modèle avec les données rééchantillonnées
num_round = 1000  # Nombre d'itérations
bst_resampled = xgb.train(params, dtrain_resampled, num_round)

# Faire des prédictions
y_pred_resampled = bst_resampled.predict(dtest)

# Évaluer le modèle
accuracy_resampled = accuracy_score(y_test, y_pred_resampled)
print(f'Accuracy after SMOTE: {accuracy_resampled:.2f}')
print(classification_report(y_test, y_pred_resampled, target_names=target_names))

Accuracy after SMOTE: 0.83
              precision    recall  f1-score   support

    Diseased       0.25      0.06      0.09        36
     Healthy       0.84      0.99      0.90       659
       Pests       0.00      0.00      0.00        40
    Stressed       0.00      0.00      0.00        54

    accuracy                           0.83       789
   macro avg       0.27      0.26      0.25       789
weighted avg       0.71      0.83      0.76       789

